# ------ **Import Subroutines and Settings** -------

# Set up for importing data ie: **df_m_ib77_nn** to add to the main data frame **df_m_ib_tst**

In [1]:
import sys
print(sys.executable)
print("note: THIS IS THE DIRECTORY PYTHON IS WORKING IN.")

/home/bhuns/miniconda3/bin/python
note: THIS IS THE DIRECTORY PYTHON IS WORKING IN.


In [2]:
# Imports required for Loading, sorting .csx files to create specific data sets ie mrn inbody readings. 
%run ./sys_funcs.py              # loads all the def functions in sys_funcs.py into memory
#import sys_funcs                 # gives access to these def function digitalform that are in memory
from pathlib import Path
import csv
import pandas as pd
import numpy as np
import tkinter as tk
import pickle
from pathlib import Path
import csv
import os
import sys
from datetime import datetime
from datetime import time
from sys_funcs import read_csv_to_array
from sys_funcs import clean_wsl_path
from sys_funcs import array_to_dt_row_dict
from sys_funcs import make_blnk_update_row_dict
from sys_funcs import transpose_csv_to_col_dict
#from sys_funcs import update_values_with_config, get_update_result
from sys_funcs import transfer_updates
from sys_funcs import get_dtv_range
from sys_funcs import universal_import
from sys_funcs import parse_inbody_timestamp
from sys_funcs import build_lut
from sys_funcs import extract_a_column_as_df
from sys_funcs import extract_multicolumns_as_df
from sys_funcs import validate_and_sort_timestamps
from sys_funcs import extract_and_filter_by_time_window
from sys_funcs import read_file_dual_path
from sys_funcs import write_file_dual_path
from sys_funcs import asc_to_csv_cnv
from collections.abc import Mapping
import re
#from sys_funcs import 

In [3]:
# set print rows  This worksheet sets maximum # of rows printed
pd.set_option('display.max_rows', 1000)  # Adjust the number of rows to display
# pd.reset_option('display.max_rows')  
print('print set to 1000 rows max' )

print set to 1000 rows max


In [4]:
print("NOTE: timestamp = Test Date / Time does not work  use computed time stamp")

NOTE: timestamp = Test Date / Time does not work  use computed time stamp


# Pre-concatinate processing import and prepare 
1. **"df_m_ib_tst"**
2. **"df_m_ib77_nn"**


## def functions called in data importing & refinment.

In [51]:
media_lst = [
    "timestamp",
    "dtv",
    "ib_id",
    "cls",
    "cmmnts"
]

In [6]:
# 3rd version def drop_duplicates_by_test_time(df, keep='first', log=True):
def drop_duplicates_by_test_time(df, keep='first', log=True):
    """
    Removes duplicate rows based on the 'Test Date / Time' column.
    Keeps only the first (or last) occurrence.
    """
    df = df.copy()

    # Identify duplicate timestamps (beyond the one we keep)
    dupes = (
        df.loc[df.duplicated(subset=['Test Date / Time'], keep=keep), 'Test Date / Time']
        .astype(str)
        .values
        .tolist()
    )

    # Drop duplicate rows
    df = df.drop_duplicates(subset=['Test Date / Time'], keep=keep)

    # Optional logging
    if log and dupes:
        print("Removed duplicate rows for timestamps:", dupes)

    return df


In [7]:
# def strip_numbers_from_columns(df):
import re

def strip_numbers_from_columns(df):
    """
    Removes leading/trailing numbers and any leftover separators
    so that cases like '1.0ID' become 'ID'.
    """
    df = df.copy()
    new_cols = {}

    for col in df.columns:
        cleaned = col

        # Remove leading numbers + separators
        cleaned = re.sub(r'^\d+[\s\-\_\.:]*', '', cleaned)

        # Remove trailing numbers + separators
        cleaned = re.sub(r'[\s\-\_\.:]*\d+$', '', cleaned)

        # Remove leftover leading/trailing punctuation (.,-_:) after number removal
        cleaned = re.sub(r'^[\.\-\_\:]+', '', cleaned)
        cleaned = re.sub(r'[\.\-\_\:]+$', '', cleaned)

        new_cols[col] = cleaned

    return df.rename(columns=new_cols)


In [8]:
# Use drop duplicate function If duplicates are found
def drop_duplicate_columns(df, keep='first', log=True):
    """
    Removes duplicate column names from a DataFrame, keeping only the first
    (or last) occurrence. Useful after column-cleaning steps that may cause
    collisions. good i'm moving this around because I want to go ahead and do the I'm talking too

    Parameters
    ----------
    df : pandas.DataFrame
        Input DataFrame.
    keep : {'first', 'last'}, default 'first'
        Which duplicate to keep.
    log : bool, default True
        Whether to print which columns were removed.

    Returns
    -------
    pandas.DataFrame
        DataFrame with duplicate columns removed.
    """
    df = df.copy()

    # Identify duplicates beyond the one we keep
    dupes = df.columns[df.columns.duplicated(keep=keep)].tolist()

    # Drop them
    df = df.loc[:, ~df.columns.duplicated(keep=keep)]

    # Optional logging
    if log and dupes:
        print("Removed duplicate columns:", dupes)

    return df


In [9]:
# def prepend_empty_columns(df, col_list):
def prepend_empty_columns(df, col_list):
    """
    Prepend empty columns (from col_list) to the front of df.
    Returns a new DataFrame.
    """
    import pandas as pd

    # Create empty columns with same row count
    empty_df = pd.DataFrame(
        {col: [None] * len(df) for col in col_list}
    )

    # Prepend them
    return pd.concat([empty_df, df], axis=1)


In [10]:
# def fill_ib_media_cols(df):
#filling the media cols
def fill_ib_media_cols(df):
    """
    Fills the 5 leading operator columns for InBody datasets:
      - timestamp  ← parsed from 'Test Date / Time' (YYYYMMDDHHMMSS)
      - dtv        ← days since 1900‑01‑01
      - ib_id      ← 'mrn' if test time 03:00–23:59, else 'eve'
      - cls        ← NaN
      - cmmnts     ← NaN
    """

    df = df.copy()

    # --- 1. timestamp (correct parsing) ------------------------
    df['timestamp'] = pd.to_datetime(
        df['Test Date / Time'].astype(str),
        format="%Y%m%d%H%M%S",
        errors="coerce"
    )

    # --- 2. dtv: days since 1900‑01‑01 -------------------------
    origin = pd.Timestamp("1900-01-01")
    df['dtv'] = (df['timestamp'] - origin).dt.days

    # --- 3. ib_id classification -------------------------------
    def classify_ib_id(ts):
        if pd.isna(ts):
            return np.nan
        hour = ts.hour
        return "mrn" if 3 <= hour <= 12 else "eve"

    df['ib_id'] = df['timestamp'].apply(classify_ib_id)

    # --- 4. cls ------------------------------------------------
    df['cls'] = np.nan

    # --- 5. cmmnts ---------------------------------------------
    df['cmmnts'] = np.nan

    return df


In [11]:
# Sort the rows by timestamp
def sort_by_timestamp(df):
    """
    Sorts an InBody dataframe by the 'timestamp' column
    in ascending chronological order.
    """
    df = df.copy()
    df = df.sort_values(by='timestamp', ascending=True)
    df = df.reset_index(drop=True)
    return df


In [12]:
# remove duplicates on the basis of timestamp
def remove_ib_duplicates(df, subset_cols=None):
    """
    Removes duplicate InBody rows based on key identifying columns.
    Default behavior: remove duplicates based on ['ID', 'timestamp'].
    """
    df = df.copy()

    # Default duplicate definition
    if subset_cols is None:
        subset_cols = ['timestamp']
        # subset_cols = ['ID', 'timestamp']
    # Remove duplicates, keeping the first occurrence
    df = df.drop_duplicates(subset=subset_cols, keep='first')

    # Reset index for cleanliness
    df = df.reset_index(drop=True)

    return df


# UPDATE **"df_m_ib_tst"** with latest data

## Bringing in the main data file from Excel and produce clean **df_mf_ib_tst_nn** 

### load from XL

In [13]:
#this brings in the main data file from Excel
df_m_ib_tst = pd.read_csv("/home/bhuns/JL_2/data/ib_tst/m_ib_tst.csv")
df_m_ib_tst_nn = strip_numbers_from_columns(df_m_ib_tst)
# verify 
df_m_ib_tst_nn
print("imported **df_m_ib_tst_nn** ie numbers removed from col heads")

imported **df_m_ib_tst_nn** ie numbers removed from col heads


### Insure no numbers in col names & slice-abilty

In [14]:
# df_m_ib_tst_nn slicing
# verify 
print('df_m_ib_tst_nn imported \n Test Date / Time \n',df_m_ib_tst_nn['Test Date / Time'])  # list form wo meta


df_m_ib_tst_nn imported 
 Test Date / Time 
 0     20251213054921
1     20251214234814
2     20251217075000
3     20251218081604
4     20251216233809
5     20251215080038
6     20251212221713
7     20251219080729
8     20251214090336
9     20251218212747
10    20251219232758
11    20251215233324
12    20251216064808
13    20251211092610
14    20251217230745
15    20251220085024
16    20251212084231
17    20251213054921
18    20251214234814
19    20251217075000
20    20251218081604
21    20251216233809
22    20251215080038
23    20251212221713
24    20251219080729
25    20251214090336
26    20251218212747
27    20251219232758
28    20251215233324
29    20251216064808
30    20251211092610
31    20251217230745
32    20251220085024
33    20251212084231
34    20251213054921
35    20251214234814
36    20251217075000
37    20251218081604
38    20251216233809
39    20251215080038
40    20251212221713
41    20251219080729
42    20251214090336
43    20251218212747
44    20251219232758
45    2025

### Insure that media_lst cols inserted in **df_m_ib_tst_nn**

In [15]:
# verify 
df_m_ib_tst_nn[media_lst]

,Test Date / Time,timestamp,dtv,ib_id,cls,cmmnts
0,20251213054921,NaN,NaN,NaN,NaN,NaN
1,20251214234814,NaN,NaN,NaN,NaN,NaN
2,20251217075000,NaN,NaN,NaN,NaN,NaN
3,20251218081604,NaN,NaN,NaN,NaN,NaN
4,20251216233809,NaN,NaN,NaN,NaN,NaN
5,20251215080038,NaN,NaN,NaN,NaN,NaN
6,20251212221713,NaN,NaN,NaN,NaN,NaN
7,20251219080729,NaN,NaN,NaN,NaN,NaN
8,20251214090336,NaN,NaN,NaN,NaN,NaN
9,20251218212747,NaN,NaN,NaN,NaN,NaN


### Insure that media_lst cols are filled in **df_m_ib_tst_nn**

In [70]:
# fill_ib_media_cols(df_m_ib_tst_nn)
df_mf_ib_tst_nn = fill_ib_media_cols(df_m_ib_tst_nn)
# verify 
df_mf_ib_tst_nn[media_lst]

,timestamp,dtv,ib_id,cls,cmmnts
0,2025-12-13 05:49:21,46002,mrn,NaN,NaN
1,2025-12-14 23:48:14,46003,eve,NaN,NaN
2,2025-12-17 07:50:00,46006,mrn,NaN,NaN
3,2025-12-18 08:16:04,46007,mrn,NaN,NaN
4,2025-12-16 23:38:09,46005,eve,NaN,NaN
5,2025-12-15 08:00:38,46004,mrn,NaN,NaN
6,2025-12-12 22:17:13,46001,eve,NaN,NaN
7,2025-12-19 08:07:29,46008,mrn,NaN,NaN
8,2025-12-14 09:03:36,46003,mrn,NaN,NaN
9,2025-12-18 21:27:47,46007,eve,NaN,NaN


## Import new data from ip77 in the data folder of repo and produce clean rows ready to concatinate

### This segment imports the data from the Excel file_ib77 to dataframe with numbers in col names

In [17]:
# loads the new from the 770
df_ib77_raw = universal_import(
    folder_path="/home/bhuns/JL_2/data/ib77",
    pattern="*"
)
# verify df_ib77_raw #print("df_ib77_raw w/o numbers OK")

✅ Loaded 091725_20251225081906.csv with utf-8
✅ Loaded 091725_20251213054921.csv with utf-8
✅ Loaded 091725_20251214234814.csv with utf-8
✅ Loaded 091725_20251223231059.csv with utf-8
✅ Loaded 091725_20251217075000.csv with utf-8
✅ Loaded 091725_20251218081604.csv with utf-8
✅ Loaded 091725_20251221071026.csv with utf-8
✅ Loaded 091725_20251216233809.csv with utf-8
✅ Loaded 091725_20251215080038.csv with utf-8
✅ Loaded 091725-1_20251212221713.csv with utf-8
✅ Loaded 091725_20251219080729.csv with utf-8
✅ Loaded 091725_20251221205127.csv with utf-8
✅ Loaded 091725_20251226083412.csv with utf-8
✅ Loaded 091725_20251227083952.csv with utf-8
✅ Loaded 091725_20251214090336.csv with utf-8
✅ Loaded 091725_20251218212747.csv with utf-8
✅ Loaded 091725_20251224080342.csv with utf-8
✅ Loaded 091725_20251222221501.csv with utf-8
✅ Loaded 091725_20251219232758.csv with utf-8
✅ Loaded 091725_20251215233324.csv with utf-8
✅ Loaded 091725_20251224234318.csv with utf-8
✅ Loaded 091725_20251216064808.c

### This segment strips off the col names of numbers and produces "df_m_ib77_nn" and demonstrates slicing

In [18]:
# verify slicing df_ib77_nn[['ID','Test Date / Time','ECW/TBW']]  #,"SMM (Skeletal Muscle Mass)" , "Weight" , "BMR (Basal Metabolic Rate)" , "ECW/TBW"]].head(20)

In [33]:
df_ib77_nn = strip_numbers_from_columns(df_ib77_raw)
#verify 
print(list(df_ib77_nn.columns))

['Name', 'ID', 'Height', 'Date of Birth', 'Age', 'Mobile Number', 'Phone Number', 'Zip Code', 'Address', 'E-mail', 'Date of Registration', 'Memo', 'Test Date / Time', 'Weight', 'TBW (Total Body Water)', 'ICW (Intracellular Water)', 'ECW (Extracellular Water)', 'DLM (Dry Lean Mass)', 'BFM (Body Fat Mass)', 'LBM (Lean Body Mass)', 'SMM (Skeletal Muscle Mass)', 'BMI (Body Mass Index)', 'PBF (Percent Body Fat)', 'LBM of Right Arm', 'LBM% of Right Arm', 'LBM of Left Arm', 'LBM% of Left Arm', 'LBM of Trunk', 'LBM% of Trunk', 'LBM of Right Leg', 'LBM% of Right Leg', 'LBM of Left Leg', 'LBM% of Left Leg', 'Leg Lean Mass', 'TBW of Right Arm', 'TBW of Left Arm', 'TBW of Trunk', 'TBW of Right Leg', 'TBW of Left Leg', 'ICW of Right Arm', 'ICW of Left Arm', 'ICW of Trunk', 'ICW of Right Leg', 'ICW of Left Leg', 'ECW of Right Arm', 'ECW of Left Arm', 'ECW of Trunk', 'ECW of Right Leg', 'ECW of Left Leg', 'ECW/TBW', 'ECW/TBW of Right Arm', 'ECW/TBW of Left Arm', 'ECW/TBW of Trunk', 'ECW/TBW of Right 

### This segment eliminates duplicates in **df_ib77_nn**

In [20]:
# Seek out and drop duplicates
df_ib77_nn = drop_duplicate_columns(df_ib77_nn)


Removed duplicate columns: ['Systolic', 'Diastolic', 'Pulse', 'Mean Artery Pressure', 'Pulse Pressure', 'Rate Pressure Product']


In [54]:
# Testing duplicates in the column titles
df_ib77_nn.columns[df_ib77_nn.columns.duplicated()]

print ("df_ib77_nn , No numbers, no duplicates OK")

df_ib77_nn , No numbers, no duplicates OK


### This segment adds **media_cols** and fills them from data in the results **"df_m_ib77_nn"**

In [55]:
df_m_ib77_nn = prepend_empty_columns(df_ib77_nn, media_lst)
# verify 
df_m_ib77_nn

,timestamp,dtv,ib_id,cls,cmmnts,Name,ID,Height,Date of Birth,Age,...,R/Ht,Xc/Ht,HGS of Left Arm 1st,HGS of Left Arm 2nd,HGS of Right Arm 1st,HGS of Right Arm 2nd,HGS/WT,Unnamed,source_file,encoding_used
0,None,None,None,None,None,91725,6ft 06.0in,1939.02.23.,F,86.0,...,0.0,0.0,-,-,-,-,-,NaN,091725_20251225081906.csv,utf-8
1,None,None,None,None,None,91725,6ft 06.0in,1939.02.23.,F,86.0,...,0.0,0.0,-,-,-,-,-,NaN,091725_20251213054921.csv,utf-8
2,None,None,None,None,None,91725,6ft 06.0in,1939.02.23.,F,86.0,...,0.0,0.0,-,-,-,-,-,NaN,091725_20251214234814.csv,utf-8
3,None,None,None,None,None,91725,6ft 06.0in,1939.02.23.,F,86.0,...,0.0,0.0,-,-,-,-,-,NaN,091725_20251223231059.csv,utf-8
4,None,None,None,None,None,91725,6ft 06.0in,1939.02.23.,F,86.0,...,0.0,0.0,-,-,-,-,-,NaN,091725_20251217075000.csv,utf-8
5,None,None,None,None,None,91725,6ft 06.0in,1939.02.23.,F,86.0,...,0.0,0.0,-,-,-,-,-,NaN,091725_20251218081604.csv,utf-8
6,None,None,None,None,None,91725,6ft 06.0in,1939.02.23.,F,86.0,...,0.0,0.0,-,-,-,-,-,NaN,091725_20251221071026.csv,utf-8
7,None,None,None,None,None,91725,6ft 06.0in,1939.02.23.,F,86.0,...,0.0,0.0,-,-,-,-,-,NaN,091725_20251216233809.csv,utf-8
8,None,None,None,None,None,91725,6ft 06.0in,1939.02.23.,F,86.0,...,0.0,0.0,-,-,-,-,-,NaN,091725_20251215080038.csv,utf-8
9,None,None,None,None,None,91725,6ft 06.0in,1939.02.23.,F,86.0,...,0.0,0.0,-,-,-,-,-,NaN,091725-1_20251212221713.csv,utf-8


### This segment eliminates duplicates in **df_m_ib77_nn**

In [57]:
df_m_ib77_nn = drop_duplicate_columns(df_m_ib77_nn)
print ("df_m_ib77_nn , No numbers, no duplicates OK")

df_m_ib77_nn , No numbers, no duplicates OK


In [64]:
# verify 
df_m_ib77_nn[media_lst]

,timestamp,dtv,ib_id,cls,cmmnts
0,None,None,None,None,None
1,None,None,None,None,None
2,None,None,None,None,None
3,None,None,None,None,None
4,None,None,None,None,None
5,None,None,None,None,None
6,None,None,None,None,None
7,None,None,None,None,None
8,None,None,None,None,None
9,None,None,None,None,None


In [66]:
df_m_ib77_nn = drop_duplicate_columns(df_m_ib77_nn)


In [67]:
df_m_ib77_nn.columns[df_m_ib77_nn.columns.duplicated()]
print("No duplicates detected")

No duplicates detected


In [62]:
df_m_ib77_nn['Test Date / Time']

0     20251225081906
1     20251213054921
2     20251214234814
3     20251223231059
4     20251217075000
5     20251218081604
6     20251221071026
7     20251216233809
8     20251215080038
9     20251212221713
10    20251219080729
11    20251221205127
12    20251226083412
13    20251227083952
14    20251214090336
15    20251218212747
16    20251224080342
17    20251222221501
18    20251219232758
19    20251215233324
20    20251224234318
21    20251216064808
22    20251211092610
23    20251223082717
24    20251217230745
25    20251226232556
26    20251220085024
27    20251222083404
28    20251212084231
29    20251226004351
30    20251220233943
Name: Test Date / Time, dtype: int64

In [69]:
# fill_ib_media_cols(df_m_ib77_nn)
df_mf_ib77_nn = fill_ib_media_cols(df_m_ib77_nn)
# verify 
df_mf_ib77_nn

,timestamp,dtv,ib_id,cls,cmmnts,Name,ID,Height,Date of Birth,Age,...,R/Ht,Xc/Ht,HGS of Left Arm 1st,HGS of Left Arm 2nd,HGS of Right Arm 1st,HGS of Right Arm 2nd,HGS/WT,Unnamed,source_file,encoding_used
0,2025-12-25 08:19:06,46014,mrn,NaN,NaN,91725,6ft 06.0in,1939.02.23.,F,86.0,...,0.0,0.0,-,-,-,-,-,NaN,091725_20251225081906.csv,utf-8
1,2025-12-13 05:49:21,46002,mrn,NaN,NaN,91725,6ft 06.0in,1939.02.23.,F,86.0,...,0.0,0.0,-,-,-,-,-,NaN,091725_20251213054921.csv,utf-8
2,2025-12-14 23:48:14,46003,eve,NaN,NaN,91725,6ft 06.0in,1939.02.23.,F,86.0,...,0.0,0.0,-,-,-,-,-,NaN,091725_20251214234814.csv,utf-8
3,2025-12-23 23:10:59,46012,eve,NaN,NaN,91725,6ft 06.0in,1939.02.23.,F,86.0,...,0.0,0.0,-,-,-,-,-,NaN,091725_20251223231059.csv,utf-8
4,2025-12-17 07:50:00,46006,mrn,NaN,NaN,91725,6ft 06.0in,1939.02.23.,F,86.0,...,0.0,0.0,-,-,-,-,-,NaN,091725_20251217075000.csv,utf-8
5,2025-12-18 08:16:04,46007,mrn,NaN,NaN,91725,6ft 06.0in,1939.02.23.,F,86.0,...,0.0,0.0,-,-,-,-,-,NaN,091725_20251218081604.csv,utf-8
6,2025-12-21 07:10:26,46010,mrn,NaN,NaN,91725,6ft 06.0in,1939.02.23.,F,86.0,...,0.0,0.0,-,-,-,-,-,NaN,091725_20251221071026.csv,utf-8
7,2025-12-16 23:38:09,46005,eve,NaN,NaN,91725,6ft 06.0in,1939.02.23.,F,86.0,...,0.0,0.0,-,-,-,-,-,NaN,091725_20251216233809.csv,utf-8
8,2025-12-15 08:00:38,46004,mrn,NaN,NaN,91725,6ft 06.0in,1939.02.23.,F,86.0,...,0.0,0.0,-,-,-,-,-,NaN,091725_20251215080038.csv,utf-8
9,2025-12-12 22:17:13,46001,eve,NaN,NaN,91725,6ft 06.0in,1939.02.23.,F,86.0,...,0.0,0.0,-,-,-,-,-,NaN,091725-1_20251212221713.csv,utf-8


### This segment eliminates duplicates based on **"test_time" in **"df_mf_ib77_nn"**

In [73]:
df_mf_ib77_nn.columns[df_mf_ib77_nn.columns.duplicated()]
print("No duplicates **df_mf_ib77_nn** detected")

No duplicates **df_mf_ib77_nn** detected


In [71]:
# verify 
df_mf_ib_tst_nn[media_lst]

,timestamp,dtv,ib_id,cls,cmmnts
0,2025-12-13 05:49:21,46002,mrn,NaN,NaN
1,2025-12-14 23:48:14,46003,eve,NaN,NaN
2,2025-12-17 07:50:00,46006,mrn,NaN,NaN
3,2025-12-18 08:16:04,46007,mrn,NaN,NaN
4,2025-12-16 23:38:09,46005,eve,NaN,NaN
5,2025-12-15 08:00:38,46004,mrn,NaN,NaN
6,2025-12-12 22:17:13,46001,eve,NaN,NaN
7,2025-12-19 08:07:29,46008,mrn,NaN,NaN
8,2025-12-14 09:03:36,46003,mrn,NaN,NaN
9,2025-12-18 21:27:47,46007,eve,NaN,NaN


### This segment sorts based on **"timestamp"** in **"df_mf_ib77_nn"**

In [74]:
#sort_by_timestamp(df_mf_ib77_nn)
df_mf_ib77_nn_s = sort_by_timestamp(df_mf_ib77_nn)
#verify 
df_mf_ib77_nn_s

,timestamp,dtv,ib_id,cls,cmmnts,Name,ID,Height,Date of Birth,Age,...,R/Ht,Xc/Ht,HGS of Left Arm 1st,HGS of Left Arm 2nd,HGS of Right Arm 1st,HGS of Right Arm 2nd,HGS/WT,Unnamed,source_file,encoding_used
0,2025-12-11 09:26:10,46000,mrn,NaN,NaN,251211-1,6ft 06.0in,1939.02.23.,M,86.0,...,0.0,0.0,-,-,-,-,-,NaN,091725-1_20251211092610.csv,utf-8
1,2025-12-12 08:42:31,46001,mrn,NaN,NaN,251211-1,6ft 06.0in,1939.02.23.,M,86.0,...,0.0,0.0,-,-,-,-,-,NaN,091725-1_20251212084231.csv,utf-8
2,2025-12-12 22:17:13,46001,eve,NaN,NaN,91725,6ft 06.0in,1939.02.23.,F,86.0,...,0.0,0.0,-,-,-,-,-,NaN,091725-1_20251212221713.csv,utf-8
3,2025-12-13 05:49:21,46002,mrn,NaN,NaN,91725,6ft 06.0in,1939.02.23.,F,86.0,...,0.0,0.0,-,-,-,-,-,NaN,091725_20251213054921.csv,utf-8
4,2025-12-14 09:03:36,46003,mrn,NaN,NaN,91725,6ft 06.0in,1939.02.23.,F,86.0,...,0.0,0.0,-,-,-,-,-,NaN,091725_20251214090336.csv,utf-8
5,2025-12-14 23:48:14,46003,eve,NaN,NaN,91725,6ft 06.0in,1939.02.23.,F,86.0,...,0.0,0.0,-,-,-,-,-,NaN,091725_20251214234814.csv,utf-8
6,2025-12-15 08:00:38,46004,mrn,NaN,NaN,91725,6ft 06.0in,1939.02.23.,F,86.0,...,0.0,0.0,-,-,-,-,-,NaN,091725_20251215080038.csv,utf-8
7,2025-12-15 23:33:24,46004,eve,NaN,NaN,91725,6ft 06.0in,1939.02.23.,F,86.0,...,0.0,0.0,-,-,-,-,-,NaN,091725_20251215233324.csv,utf-8
8,2025-12-16 06:48:08,46005,mrn,NaN,NaN,91725,6ft 06.0in,1939.02.23.,F,86.0,...,0.0,0.0,-,-,-,-,-,NaN,091725_20251216064808.csv,utf-8
9,2025-12-16 23:38:09,46005,eve,NaN,NaN,91725,6ft 06.0in,1939.02.23.,F,86.0,...,0.0,0.0,-,-,-,-,-,NaN,091725_20251216233809.csv,utf-8


# WORKING cell 
1. Verify this Timestamp has not been recorded
2. If not insert value col by col

#### Calculate excess columns in the main data frame compared the columns in the IB77 data frame.

In [ ]:
# Verify that we have the main data frame
df_m_ib_tst.columns

In [ ]:
#Verify that we have data from data frame to be added to the main data frame 
# If it's not there read it from pickle
# verify df_ib77_raw.columns

In [ ]:
# READ the IB77 Columns without numbers FROM pickle 
with open("df_ib77_raw_nms.pkl", "rb") as f:
    df_ib77_raw_nms = pickle.load(f)
# Verify df_ib77_raw_nms
print ("from pkl df_ib77_raw_nms.pkl OK")

In [ ]:
# Load the IB77 Columns without numbers to pickle 
with open("df_ib77_raw_nms.pkl", "wb") as f:
    pickle.dump(df_ib77_raw_nms, f)
print ("df_ib77_raw_nms.pkl OK")

## Read the latest data from ib77

### ADD MEDA , CLEAN AND SORT  >>>>   **df_m_ib_tst_nn**

## Fill in the meta columns by reading the data in the rest of the columns >>> **df_m_ib_tst_nn**

In [ ]:
# verify df_m_ib_tst_nn[media_lst].head(20)

## Sort the data frame on the basis of timestamp you >> **df_m_ib_tst_s**

In [ ]:
df_m_ib_tst_nn_s = sort_by_timestamp(df_m_ib_tst_nn)

In [ ]:
# verify 
df_m_ib_tst_nn_s[media_lst].head(20)

In [ ]:
#verify 
df_m_ib_tst_nn_s[["dtv", "ib_id" , "SMM (Skeletal Muscle Mass)" , "Weight" , "BMR (Basal Metabolic Rate)" , "ECW/TBW"]].head(20)

## Remove duplicates [...xd]  Retaining the first of a duplicate pair  >>> **df_m_ib_tst_s_xd**

In [ ]:
df_m_ib_tst_nn_s_xd  =  remove_ib_duplicates(df_m_ib_tst_s, subset_cols=None)
# verify df_m_ib_tst_nn_s_xd

## Slicing the fully processed data frame [df_m_ib_tst_nn_s_xd] to display columns of interest  

In [ ]:
#verify 
df_m_ib_tst_nn_s_xd[["dtv","timestamp","SMM (Skeletal Muscle Mass)" , "Weight" , "BMR (Basal Metabolic Rate)" , "ECW/TBW"]].head(20)

# The stored data is loaded checked and verified, and ready to receive new data
1. **"df_m_ib_tst"**
2. **"df_m_ib_tst_nms"**
3. **"df_ib97_tst"**
4. **"df_ib97_tst_nms"**
5. **"df_ib77_tst"**
6. **"df_ib77_tst_nms"**
7. **"_______________"**

In [ ]:
df_m_ib_tst_nms = list(df_m_ib_tst.columns)
#df_ib77_raw_nms with numbers 
#verify  df_m_ib_tst_nms
print("df_m_ib_tst_nms w/o numbers OK")

In [ ]:
# Testing duplicates in the column titles
df_m_ib_tst.columns[df_m_ib_tst.columns.duplicated()]
print("df_m_ib_tst_nms w/o numbers no duplicate OK")

# This will load the ib77 into the accumulated data

In [ ]:
# verify df_m_ib_tst
print(" The main df_m_ib_tst is in place with col numbers:")

In [ ]:
# verify df_m_ib_tst_nn
df_m_ib_tst_nn = strip_numbers_from_columns(df_m_ib_tst)
df_m_ib_tst_nn
print(" The main df_m_ib_tst_nn is in place w/o col numbers:")

In [ ]:
# This is the IB77 columns that still have the numbers attached
ib77_raw_missing_cols_n = [col for col in ib77_raw.columns 
                           if col not in df_m_ib_tst_nn.columns]

# verify ib77_raw_missing_cols

In [ ]:
for col in ib77_raw_missing_cols:
    ib77_raw[col] = np.nan
ib77_raw              # ["ECW/TBW"]

In [ ]:
import pandas as pd

# Suppose missing_cols is a list of column names you want to add
new_cols = pd.DataFrame({col: [None] * len(ib77_raw) for col in missing_cols})
new_cols

In [ ]:
# Add them in one operation
ib77_raw = pd.concat([ib77_raw, new_cols], axis=1)

In [ ]:
# Add any missing columns to IB77_raw
for col in df_m_ib_tst.columns:
    if col not in ib77_raw.columns:
        ib77_raw[col] = None
# verify 
ib77_raw

In [ ]:
df_ib77_raw[["BMR (Basal Metabolic Rate)" , "ECW/TBW"]].head(20)

In [ ]:
# The main data frame is larger than the add data frame so we need To determine the Missing columns.
missing_cols = [c for c in df_m_ib_tst.columns if c not in df_ib77_raw.columns]
#verify missing_cols

In [ ]:
# Use the missing_cols dataframe in order add columns with NAn values to the ib77 values so it can be appended to with the main data frame "df_m_ib_tst
df_ib77_cncnt = df_ib77_raw.copy()    # start building the df
#verify df_ib77_cncnt

In [ ]:
# 1. Add missing columns
for col in missing_cols:
    df_ib77_cncnt[col] = np.nan
#verify df_ib77_cncnt

In [ ]:
# 3. Reorder to match master schema
df_ib77_cncnt = df_ib77_cncnt[df_m_ib_tst.columns]
#verify df_ib77_cncnt

In [ ]:
# 4. Append
df_m_ib_tst = pd.concat([df_m_ib_tst, df_ib77_cncnt], ignore_index=True)
#verify df_m_ib_tst

In [ ]:
# Calculate missing columns when df_ib77 raw data is appended to IB_tst Data frame
df_ib77_raw_missing_cols_n = [col for col in df_ib77_raw.columns if col not in df_m_ib_tst.columns]
# verify 
df_ib77_raw_missing_cols_n
print ("df_ib77_raw_missing_cols_n with numbers  OK")

In [ ]:
# strip numbers off So that the identical names in ib77 are the same as 97
df_ib77_raw_missing_cols_nms = strip_col_numbers(df_ib77_raw_missing_cols_n)
# Verify 
df_ib77_raw_missing_cols_nms
print ("df_ib77_raw_missing_cols_nms w/o numbers  OK")

In [ ]:
# Load the df_ib77_raw_missing_cols_nms to pickle 
with open("df_ib77_raw_missing_cols_nms.pkl", "wb") as f:
    pickle.dump(df_ib77_raw_missing_cols_nms, f)
print ("df_ib77_raw_missing_cols_nms.pkl OK")

In [ ]:
# READ thedf_ib77_raw_missing_cols_nms FROM pickle 
with open("df_ib77_raw_missing_cols_nms.pkl", "rb") as f:
    df_ib77_raw_missing_cols_nms = pickle.load(f)
# Verify df_ib77_raw_missing_cols_nms
print ("from pkl df_ib77_raw_missing_cols_nms.pkl OK")

In [ ]:
# Calculate missing columns when df_ib77 raw data is appended to ib_tst DataFrame

## Calculate the **ib_tst** columns that are not used when **ib77** is concatenated =  **xcss77**

### ib97:  This segment Reads the data from the excel file_ib97 and computes names of the column head, strips them of numbers and Puts them in a list and records them in a pickle. *ib97_raw_nms*

In [ ]:
# loads the new from the 970
ib97_raw = universal_import(
    folder_path="/home/bhuns/JL_2/data/ib97", 
    pattern="*"
)

In [ ]:
ib97_raw_nms = list(ib97_raw.columns)
# Verify 
ib97_raw_nms

In [ ]:
# strip numbers off So that the identical names in ib97 are the same as ib97
ib97_raw_nms = strip_col_numbers(ib97_raw)
# Verify
ib97_raw_nms

In [ ]:
# Load the IB97 Columns without numbers to pickle 
with open("ib97_raw_nms.pkl", "wb") as f:
    pickle.dump(ib97_raw_nms, f)


In [ ]:
# READ the IB97 Columns without numbers to pickle 
with open("ib97_raw_nms.pkl", "rb") as f:
    ib97_raw_nms = pickle.load(f)
# Verify ib97_raw_nms

#### The following data are loaded and ready to be consolidated into an updated Total data data frame "m_bi_tst" from :XL >>> "df_m_bi_tst" and new"df_ib97_raw",new "df_ib77_raw"

### This will load the ib97 into the accumulated data

In [ ]:
# Add any missing columns to IB97_raw
for col in m_ib_tst.columns:
    if col not in ib97_raw.columns:
        ib97_raw[col] = None
# Reorder IB97_raw columns to match m_ib_tst
ib97_raw = ib97_raw[m_ib_tst.columns]


In [ ]:
m_ib_tst = pd.concat([m_ib_tst, ib97_raw], ignore_index=True)
m_ib_tst

In [ ]:
# Reorder ib77_raw columns to match df_m_ib_tst
ib77_raw = ib77_raw[df_m_ib_tst.columns]

In [ ]:
# This concatenates 770 new data with the existing data That contains the accumulator data from 770 and 970
df_m_ib_tst = pd.concat([df_m_ib_tst, ib77_raw], ignore_index=True)
#verify df_m_ib_tst

In [ ]:
# This checks to see if column names are still all unique after the new data
df_m_ib_tst.columns[df_m_ib_tst.columns.duplicated()]
print(" No duplicates found Afternoon data was appended")

In [ ]:
df_m_ib_tst["

In [ ]:
ib77_raw.columns[ib77_raw.columns.duplicated()]


In [ ]:
df_m_ib_tst_s_xd[["dtv","SMM (Skeletal Muscle Mass)" , "Weight" , "BMR (Basal Metabolic Rate)" , "ECW/TBW"]].head(20)

# load latest [m_ib_tst] to xl

In [ ]:
# load latest [m_ib_tst] from_xl
folder_path = "/home/bhuns/JL_2/data/ib_tst/m_ib_tst.csv"
m_ib_tst.to_csv(folder_path, index=False)
# verify
#
m_ib_tst

# Everything below this line is residual the days the template was developed and completed. Erase it when everything using the template is completed
# ================================================================================================================================================================

# Startup only Create  **df_ib_fls_tmplt**  ie  *[COL_NMS = meta+ib770dat + ib970dat]*

### Both the 77 and the 90 test datasets are convrted to dfs In order to determine *df_ib_tst_nms* Which is a single column data frame that all of the column names of the *df_ib_tst*

#### ✅ Concatenate → Preserve Order → Drop Duplicates

In [ ]:
####  Concatenate → Preserve Order → Drop Duplicates mmmmmmmmmnnnnnnnllllll
# Concatenate in the required order
df_ib_tst_nms = pd.concat(
    [df_ib77_raw_nms, df_ib97_raw_nms],
    axis=0,
    ignore_index=True
)
# print(df_ib_tst_nms)
# Remove duplicate rows, keeping the first occurrence (from df_ib77_raw_nms)
df_ib_tst_nms = df_ib_tst_nms.drop_duplicates(keep="first")
# verify 
# print(df_ib_tst_nms)

In [ ]:
# Verifies that the duplicates have been removed and that the list has been compacted and there are not duplicates of 770 and 970 Col
df_ib_tst_nms[df_ib_tst_nms.duplicated()]

### *df_ib_tst_nms* must be converted to a simple list in order to made equal to the column heads of df_tst

In [ ]:
# The large test data frame that will be used to accept values of each from both 770 N 970 will be built on the basis of the df_ib_tst_nms, but this data bugs be in form of a strings.
# df['colname'].astype(str).tolist       # sign This is the sample given from copilot that is modified below for my situation 

ib_tst_lst = df_ib_tst_nms[0].astype(str).tolist()
# verify 
# print(ib_tst_lst)

In [ ]:
# this makes the template for the ib_tst data For both the 770 and the 970 It is made up of a 770 and a 970 with duplicates eliminated in a list of strings

df_ib_tst = pd.DataFrame(
    data = [[""] * len(df_ib_tst_lst)] * 2,
    columns = ib_tst_lst
)
#df_ib_tst_nms
df_ib_tst

In [ ]:
# Tests 
df_ib_tst["ID"]


In [ ]:
df_ib_tst.columns


In [ ]:
df_ib_tst["ID"]

## data frame of the main data storage_“ib_tst"

###  Make the edits to the structure of the main data spreadsheet that represents structure

### Then the CSV file into the worksheet to display the changes in Jupiter lab

In [ ]:
df_loaded = pd.read_csv("/home/bhuns/JL_2/data/ib_tst/ib_tst.csv")
df_loaded.columns

In [ ]:
folder_path = "/home/bhuns/JL_2/data/ib_tst/ib_tst.csv"
df_ib_tst.to_csv(folder_path, index=False)
df_ib_tst

In [ ]:
df_loaded["Age"]



In [ ]:
#==================================================================================================================== Oh

###


# Align the columns via index suggestions from Copilot to insert new data into the major data frame

Put the creation of a data frame into its own worksheet

# ==============================================================================================================================================

In [ ]:
# def strip_numbers_from_columns(df):
import re
def strip_numbers_from_columns(df):
    """
    Removes leading or trailing numbers (and optional separators)
    from column names without altering descriptor text.
    Examples cleaned:
      '1 Weight' → 'Weight'
      'Weight 1' → 'Weight'
      '2-Height' → 'Height'
      '3_TBW' → 'TBW'
      '4:LBM' → 'LBM'
    """
    df = df.copy()
    new_cols = {}

    for col in df.columns:
        # Remove leading numbers + separators
        cleaned = re.sub(r'^\d+[\s\-\_:]*', '', col)
        # Remove trailing numbers + separators
        cleaned = re.sub(r'[\s\-\_:]*\d+$', '', cleaned)
        new_cols[col] = cleaned

    df = df.rename(columns=new_cols)
    return df


In [ ]:
media_lst = [
    "Test Date / Time",
    "timestamp",
    "dtv",
    "ib_id",
    "cls",
    "cmmnts"
]
